In [1]:
import numpy as np
import pandas as pd
import pymysql
import warnings
import datetime
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('max_columns',None)
pd.set_option('max_rows',500)
pd.set_option('max_colwidth',200)

In [3]:
now_time = datetime.datetime.now().strftime('%Y%m%d')
con = pymysql.connect(host='localhost',user='root',passwd='simon',port=3306,db='db_rzq')
sql = 'select * from consumedata where (convert(data_flag,signed)-{})<365 order by data_flag;'.format(int(now_time))
sql1 = 'select 客户名称,min(convert(data_flag,signed)) 首销日 from consumedata group by 客户名称 order by 首销日; '

In [4]:
df1 = pd.read_sql(sql,con=con)
df2 = pd.read_sql(sql1,con=con)
df2.head(10)

,客户名称,首销日
0,中国共产党郑州市委员会宣传部,20200101
1,信阳市浉河区人民政府办公室,20200101
2,华夏天下知识产权运营有限公司,20200101
3,博凯健康产业有限公司,20200101
4,博凯药业有限公司,20200101
5,和信证券投资咨询股份有限公司,20200101
6,大宋官窑股份有限公司,20200101
7,巩义市嵩顶安泰旅游服务有限公司,20200101
8,建业住宅集团(中国)有限公司,20200101
9,开封市互联网信息办公室,20200101


In [5]:
endtime = datetime.datetime.now()-datetime.timedelta(days=120)
endtime = endtime.strftime('%Y%m%d')
endtime

'20211229'

In [15]:
df3 = df2[df2['首销日']>int(endtime)].reset_index(drop=True)
df3['首销日'] = pd.to_datetime(df3['首销日'],format='%Y%m%d')
df3.head(10)

,客户名称,首销日
0,中共郑州市委网络安全和信息化委员会办公室,2021-12-30
1,河南兴松环保科技有限公司,2021-12-30
2,河南晨柒网络科技有限公司,2021-12-31
3,河南晟和祥实业有限公司,2022-01-01
4,河南正商置业有限公司,2022-01-01
5,河南灏宇数字包装产业园开发有限公司,2022-01-01
6,河南畅田信息科技有限公司,2022-01-01
7,郑州开祥智能传感园区建设有限公司,2022-01-01
8,郑州商业中等专业学校,2022-01-20
9,周锐博,2022-02-08


In [12]:
df_consume = df1[['data_flag','客户名称','总消']].groupby(by=['data_flag','客户名称']).sum().reset_index()
df_consume

,data_flag,客户名称,总消
0,20200101,中国共产党郑州市委员会宣传部,4293.82
1,20200101,信阳市浉河区人民政府办公室,263.00
2,20200101,华夏天下知识产权运营有限公司,2.37
3,20200101,博凯健康产业有限公司,2316.39
4,20200101,博凯药业有限公司,1121.93
...,...,...,...
86576,20220427,郑州路捷交通设施有限公司,204.08
86577,20220427,郑州车来顺大数据科技有限公司,799.99
86578,20220427,郑州达人视界职业培训学校,743.39
86579,20220427,郑州阿生哥餐饮管理有限公司,367.75


In [8]:
new_order = pd.read_excel('../data/对应关系.xlsx')
fresh_consume = new_order[['公司名称','部门','对应销售','FEED客服','备注']]
fresh_consume = fresh_consume[fresh_consume['备注']>=endtime]
fresh_consume

,公司名称,部门,对应销售,FEED客服,备注
42,郑州欧凯龙家居有限公司,家居3C组,齐朋超,陈双,2022-03-10
733,河南兴松环保科技有限公司,综合开发组,朱祥,NaN,2021-12-30
735,河南晨柒网络科技有限公司,商服家居组,谢继凤,NaN,2021-12-31
736,郑州商业中等专业学校,综合开发组,吴家骥,NaN,2022-01-20
737,效律网-zz6,律师组,卢蒙恩,NaN,2022-01-20
738,周锐博,律师组,卢蒙恩,NaN,2022-02-08
749,河南卓倩网络科技有限公司,商服家居组,谢继凤,NaN,2022-02-11
750,河南滳锦律师事务所,律师组,王冬,NaN,2022-02-11
751,河南千维法律咨询服务有限公司南阳分公司,律师组,卢蒙恩,NaN,2022-02-14
752,郑州华东电缆有限公司,快消组,荣志昆,NaN,2022-02-17


In [13]:
df_consume['data_flag'] = pd.to_datetime(df_consume['data_flag'],format='%Y%m%d')
df_consume

,data_flag,客户名称,总消
0,2020-01-01,中国共产党郑州市委员会宣传部,4293.82
1,2020-01-01,信阳市浉河区人民政府办公室,263.00
2,2020-01-01,华夏天下知识产权运营有限公司,2.37
3,2020-01-01,博凯健康产业有限公司,2316.39
4,2020-01-01,博凯药业有限公司,1121.93
...,...,...,...
86576,2022-04-27,郑州路捷交通设施有限公司,204.08
86577,2022-04-27,郑州车来顺大数据科技有限公司,799.99
86578,2022-04-27,郑州达人视界职业培训学校,743.39
86579,2022-04-27,郑州阿生哥餐饮管理有限公司,367.75


In [16]:
data_list=[]
for i in range(df3.shape[0]):
    data = df_consume[(df_consume['客户名称']==df3.loc[i,'客户名称'])&(df_consume['data_flag']>df3.loc[i,'首销日'])].groupby(by='客户名称').agg({'data_flag':['count',np.max],
                                                                                                              '总消':np.sum})
    temp = data.reset_index().values.tolist()
    data_list.append(temp[0] if len(temp)>0 else [0]*4)
df4 = pd.DataFrame(data_list,columns=['公司名称','有消费天数','最后消费日','总消'])
df4.head(10)

,公司名称,有消费天数,最后消费日,总消
0,中共郑州市委网络安全和信息化委员会办公室,2,2022-02-22 00:00:00,175930.00
1,河南兴松环保科技有限公司,89,2022-04-27 00:00:00,17843.15
2,河南晨柒网络科技有限公司,57,2022-04-21 00:00:00,272770.81
3,河南晟和祥实业有限公司,13,2022-02-23 00:00:00,53560.99
4,河南正商置业有限公司,116,2022-04-27 00:00:00,146749.28
5,河南灏宇数字包装产业园开发有限公司,77,2022-04-01 00:00:00,19977.58
6,河南畅田信息科技有限公司,92,2022-04-18 00:00:00,35296.42
7,郑州开祥智能传感园区建设有限公司,52,2022-03-18 00:00:00,570.12
8,郑州商业中等专业学校,90,2022-04-27 00:00:00,84950.96
9,周锐博,65,2022-04-18 00:00:00,308761.22


In [17]:
df4['有消费日均'] = df4['总消']/df4['有消费天数']
df4.head(10)

,公司名称,有消费天数,最后消费日,总消,有消费日均
0,中共郑州市委网络安全和信息化委员会办公室,2,2022-02-22 00:00:00,175930.00,87965.000000
1,河南兴松环保科技有限公司,89,2022-04-27 00:00:00,17843.15,200.484831
2,河南晨柒网络科技有限公司,57,2022-04-21 00:00:00,272770.81,4785.452807
3,河南晟和祥实业有限公司,13,2022-02-23 00:00:00,53560.99,4120.076154
4,河南正商置业有限公司,116,2022-04-27 00:00:00,146749.28,1265.080000
5,河南灏宇数字包装产业园开发有限公司,77,2022-04-01 00:00:00,19977.58,259.449091
6,河南畅田信息科技有限公司,92,2022-04-18 00:00:00,35296.42,383.656739
7,郑州开祥智能传感园区建设有限公司,52,2022-03-18 00:00:00,570.12,10.963846
8,郑州商业中等专业学校,90,2022-04-27 00:00:00,84950.96,943.899556
9,周锐博,65,2022-04-18 00:00:00,308761.22,4750.172615


In [18]:
df5 = pd.merge(df4,fresh_consume,on='公司名称',how='left')
df5

,公司名称,有消费天数,最后消费日,总消,有消费日均,部门,对应销售,FEED客服,备注
0,中共郑州市委网络安全和信息化委员会办公室,2,2022-02-22 00:00:00,175930.00,87965.000000,NaN,NaN,NaN,NaT
1,河南兴松环保科技有限公司,89,2022-04-27 00:00:00,17843.15,200.484831,综合开发组,朱祥,NaN,2021-12-30
2,河南晨柒网络科技有限公司,57,2022-04-21 00:00:00,272770.81,4785.452807,商服家居组,谢继凤,NaN,2021-12-31
3,河南晟和祥实业有限公司,13,2022-02-23 00:00:00,53560.99,4120.076154,NaN,NaN,NaN,NaT
4,河南正商置业有限公司,116,2022-04-27 00:00:00,146749.28,1265.080000,NaN,NaN,NaN,NaT
5,河南灏宇数字包装产业园开发有限公司,77,2022-04-01 00:00:00,19977.58,259.449091,NaN,NaN,NaN,NaT
6,河南畅田信息科技有限公司,92,2022-04-18 00:00:00,35296.42,383.656739,NaN,NaN,NaN,NaT
7,郑州开祥智能传感园区建设有限公司,52,2022-03-18 00:00:00,570.12,10.963846,NaN,NaN,NaN,NaT
8,郑州商业中等专业学校,90,2022-04-27 00:00:00,84950.96,943.899556,综合开发组,吴家骥,NaN,2022-01-20
9,周锐博,65,2022-04-18 00:00:00,308761.22,4750.172615,律师组,卢蒙恩,NaN,2022-02-08


In [19]:
df_clean = df5.dropna(subset=['部门'],axis=0)
df_clean.drop(columns=['FEED客服'],inplace=True)
df_clean

,公司名称,有消费天数,最后消费日,总消,有消费日均,部门,对应销售,备注
1,河南兴松环保科技有限公司,89,2022-04-27 00:00:00,17843.15,200.484831,综合开发组,朱祥,2021-12-30
2,河南晨柒网络科技有限公司,57,2022-04-21 00:00:00,272770.81,4785.452807,商服家居组,谢继凤,2021-12-31
8,郑州商业中等专业学校,90,2022-04-27 00:00:00,84950.96,943.899556,综合开发组,吴家骥,2022-01-20
9,周锐博,65,2022-04-18 00:00:00,308761.22,4750.172615,律师组,卢蒙恩,2022-02-08
10,河南卓倩网络科技有限公司,56,2022-04-27 00:00:00,590312.22,10541.289643,商服家居组,谢继凤,2022-02-11
11,河南滳锦律师事务所,21,2022-03-04 00:00:00,1223.79,58.275714,律师组,王冬,2022-02-11
12,河南千维法律咨询服务有限公司南阳分公司,1,2022-02-15 00:00:00,6223.05,6223.050000,律师组,卢蒙恩,2022-02-14
13,郑州华东电缆有限公司,69,2022-04-27 00:00:00,120211.27,1742.192319,快消组,荣志昆,2022-02-17
14,河南云盛知识产权代理有限公司,64,2022-04-27 00:00:00,19877.06,310.579062,商服家居组,侯利明,2022-02-19
15,郑州鑫商汇商业管理有限公司,47,2022-04-14 00:00:00,11423.37,243.050426,综合开发组,王萌萌,2022-02-19


In [20]:
df_clean['bins'] = pd.cut(df_clean['有消费日均'],bins=[0,500,1000,3000,100000],labels=['0.5K-','0.5K-1K','1K-3K','3K+'])
df_clean

,公司名称,有消费天数,最后消费日,总消,有消费日均,部门,对应销售,备注,bins
1,河南兴松环保科技有限公司,89,2022-04-27 00:00:00,17843.15,200.484831,综合开发组,朱祥,2021-12-30,0.5K-
2,河南晨柒网络科技有限公司,57,2022-04-21 00:00:00,272770.81,4785.452807,商服家居组,谢继凤,2021-12-31,3K+
8,郑州商业中等专业学校,90,2022-04-27 00:00:00,84950.96,943.899556,综合开发组,吴家骥,2022-01-20,0.5K-1K
9,周锐博,65,2022-04-18 00:00:00,308761.22,4750.172615,律师组,卢蒙恩,2022-02-08,3K+
10,河南卓倩网络科技有限公司,56,2022-04-27 00:00:00,590312.22,10541.289643,商服家居组,谢继凤,2022-02-11,3K+
11,河南滳锦律师事务所,21,2022-03-04 00:00:00,1223.79,58.275714,律师组,王冬,2022-02-11,0.5K-
12,河南千维法律咨询服务有限公司南阳分公司,1,2022-02-15 00:00:00,6223.05,6223.050000,律师组,卢蒙恩,2022-02-14,3K+
13,郑州华东电缆有限公司,69,2022-04-27 00:00:00,120211.27,1742.192319,快消组,荣志昆,2022-02-17,1K-3K
14,河南云盛知识产权代理有限公司,64,2022-04-27 00:00:00,19877.06,310.579062,商服家居组,侯利明,2022-02-19,0.5K-
15,郑州鑫商汇商业管理有限公司,47,2022-04-14 00:00:00,11423.37,243.050426,综合开发组,王萌萌,2022-02-19,0.5K-


In [21]:
pd.pivot_table(df_clean,values='有消费日均',index='部门',columns='bins',aggfunc='count',fill_value=0)

bins,0.5K-,0.5K-1K,1K-3K,3K+
部门,,,,
KA,1,0,0,0
商服家居组,4,0,1,2
地产组,0,0,0,1
律师组,2,3,1,2
快消组,0,0,1,0
综合开发组,5,3,2,0


In [22]:
pd.pivot_table(df_clean,values='有消费日均',index='部门',columns='bins',aggfunc='mean',fill_value=0)

bins,0.5K-,0.5K-1K,1K-3K,3K+
部门,,,,
KA,365.624595,0.000000,0.000000,0.000000
商服家居组,339.749309,0.000000,2806.853929,7663.371225
地产组,0.000000,0.000000,0.000000,3213.895294
律师组,29.507857,835.535737,2489.071667,5486.611308
快消组,0.000000,0.000000,1742.192319,0.000000
综合开发组,195.003765,755.150078,1902.138363,0.000000


### 日均消费在1K以下的客户

In [24]:
df_clean[df_clean['有消费日均']<1000].sort_values(by='部门')

,公司名称,有消费天数,最后消费日,总消,有消费日均,部门,对应销售,备注,bins
24,河南固美特材料科技有限公司,37,2022-04-27 00:00:00,13528.11,365.624595,KA,李哲,2022-03-21,0.5K-
14,河南云盛知识产权代理有限公司,64,2022-04-27 00:00:00,19877.06,310.579062,商服家居组,侯利明,2022-02-19,0.5K-
16,郑州市凯尚达餐饮管理咨询有限公司,23,2022-03-18 00:00:00,9748.83,423.862174,商服家居组,齐朋超,2022-02-23,0.5K-
18,河南智诺建筑工程有限公司,45,2022-04-27 00:00:00,6622.02,147.156000,商服家居组,侯利明,2022-03-01,0.5K-
20,镇平县辰轩玉器店,6,2022-03-20 00:00:00,2864.40,477.400000,商服家居组,谢继凤,2022-03-14,0.5K-
21,马鹏程,37,2022-04-27 00:00:00,29353.86,793.347568,律师组,李楠,2022-03-14,0.5K-1K
25,河南晖鼎律师事务所,28,2022-04-27 00:00:00,25512.19,911.149643,律师组,卢蒙恩,2022-03-21,0.5K-1K
37,河南银隆律师事务所,2,2022-04-27 00:00:00,1.48,0.740000,律师组,王冬,2022-04-21,0.5K-
38,石田田,1,2022-04-27 00:00:00,802.11,802.110000,律师组,李楠,2022-04-26,0.5K-1K
11,河南滳锦律师事务所,21,2022-03-04 00:00:00,1223.79,58.275714,律师组,王冬,2022-02-11,0.5K-
